**Tabla de contenido**

- [Technical requirements](#Technical-requirements)
- [Architecting systems](#Architecting-systems)
    - [Explorando la efectividad poco razonable de los patrones](#Explorando-la-efectividad-poco-razonable-de-los-patrones)
    - [Swimming in data lakes](#Swimming-in-data-lakes)
    - [Microservices](#Microservices)
    - [Diseños basados en eventos](#Disenos-basados-en-eventos)
    - [Batching](#Batching)
    - [Containerizing](#Containerizing)
- [Pushing to ECR](#Pushing-to-ECR)
- [Hosting on ECS](#Hosting-on-ECS)
- [Pipelining 2.0](#Pipelining-2.0)
    - [Airflow](#Airflow)
    - [Airflow on AWS](#Airflow-on-AWS)
    - [Revisitar CI/CD](#Revisitar-CI/CD)

En este capítulo, profundizaremos en algunos conceptos importantes sobre la implementación de tu solución de Aprendizaje Automático (ML). Comenzaremos a cerrar el círculo del ciclo de vida del desarrollo de ML y sentaremos las bases para llevar tus soluciones al mundo.

El acto de desplegar software, de llevarlo de una demostración que puedes mostrar a algunos interesados a un servicio que, en última instancia, impactará a clientes o colegas, es un ejercicio muy emocionante pero a menudo desafiante. También sigue siendo uno de los aspectos más difíciles de cualquier proyecto de ML y hacerlo bien puede marcar la diferencia entre generar valor o simplemente crear expectativa.

Vamos a explorar algunos de los conceptos principales que ayudarán a tu equipo de ingeniería de ML a cruzar el abismo entre un divertido prototipo y soluciones que puedan ejecutarse en una infraestructura escalable de manera automatizada.

En este capítulo, cubriremos los siguientes temas:

- Arquitectura de sistemas
- Explorando la eficacia irracional de los patrones
- Contenerización
- Alojar su propio microservicio en Amazon Web Services (AWS)
- Pipelining 2.0

La siguiente sección discutirá cómo podemos arquitectar y diseñar nuestros sistemas de ML teniendo en cuenta el despliegue. ¡Vamos!

# Technical requirements

Para trabajar a través de los ejemplos en este capítulo, necesitamos que las siguientes herramientas estén instaladas:

- AWS CLI
- Postman
- Docker

# Architecting systems

No importa cómo estés trabajando para construir tu software, siempre es importante tener un diseño en mente. Esta sección destacará las consideraciones clave que debemos tener en cuenta al arquitectar sistemas de aprendizaje automático.

Considera un escenario en el que estás contratado para organizar la construcción de una casa. No simplemente iríamos a contratar a un equipo de constructores, comprar todos los suministros, alquilar todo el equipo y simplemente decirles a todos que empiecen a construir. Tampoco asumiríamos que sabemos exactamente lo que el cliente que nos contrató quiere sin hablar primero con él.

En cambio, probablemente intentaríamos entender en detalle lo que el cliente quería, y luego intentar diseñar la solución que se ajustara a sus requisitos. Potencialmente, podríamos iterar este plan varias veces con ellos y con expertos apropiados que conocieran los detalles de las piezas que alimentan el diseño general. Aunque no estamos interesados en construir casas (o tal vez lo estés, ¡pero no habrá ninguna en este libro!), aún podemos ver la analogía con el software. Antes de construir cualquier cosa, debemos crear un diseño efectivo y claro. Este diseño proporciona la dirección de viaje para la solución y ayuda al equipo de construcción a saber exactamente en qué componentes trabajarán. Esto significa que estaremos seguros de que lo que construimos resolverá el problema del usuario final.

Esto, en resumen, es de lo que se trata la arquitectura de software.

Si hiciéramos el equivalente del ejemplo anterior para nuestra solución de ML, algunas de las siguientes cosas podrían suceder. Podríamos terminar con una base de código muy confusa, con algunos ingenieros de ML en nuestro equipo construyendo elementos y funcionalidades que ya están cubiertos por el trabajo que han realizado otros ingenieros. También podríamos construir algo que fundamentalmente no pueda funcionar más adelante en el proyecto; por ejemplo, si hemos seleccionado una herramienta que tiene requisitos ambientales específicos que no podemos cumplir debido a otro componente. También podríamos tener dificultades para anticipar qué infraestructura necesitamos tener provisionada con anticipación, lo que llevaría a una carrera desorganizada dentro del proyecto para obtener el recurso correcto. También podríamos subestimar la cantidad de trabajo requerido y perder nuestra fecha límite. Todos estos son resultados que deseamos evitar y se pueden evitar si seguimos un buen diseño.

Para ser efectiva, la arquitectura de una pieza de software debe proporcionar al menos las siguientes cosas al equipo que trabaja en la construcción de la solución:

- Debería definir los componentes funcionales necesarios para resolver el problema en su totalidad.
- Debería definir cómo interactuarán estos componentes funcionales, generalmente a través del intercambio de alguna forma de datos.
- Debería mostrar cómo la solución puede ampliarse en el futuro para incluir funcionalidades adicionales que el cliente pueda requerir.
- Debería proporcionar orientación sobre qué herramientas se deben seleccionar para implementar cada uno de los componentes descritos en la arquitectura.
- Debería estipular el flujo del proceso para la solución, así como el flujo de datos.

Esto es lo que debería hacer una buena pieza de arquitectura, pero ¿qué significa esto en la práctica?

No existe una definición estricta de cómo debe compilarse una arquitectura. El punto clave es que actúa como un diseño contra el cual se puede avanzar en la construcción. Así que, por ejemplo, esto podría tomar la forma de un bonito diagrama con cuadros, líneas y algo de texto, o podría ser un documento de varias páginas. Podría compilarse utilizando un lenguaje de modelado formal como el Lenguaje de Modelado Unificado (UML), o no.

Esto a menudo depende del contexto empresarial en el que operas y de los requisitos que se imponen a las personas que escriben la arquitectura. La clave es que marque los puntos mencionados anteriormente y brinde a los ingenieros una orientación clara sobre qué construir y cómo todo se unirá.

La arquitectura es un tema vasto y fascinante en sí mismo, por lo que no profundizaremos mucho más en los detalles de esto aquí, pero ahora nos centraremos en lo que significa la arquitectura en un contexto de ingeniería de aprendizaje automático.

## Explorando la efectividad poco razonable de los patrones

En este libro, ya hemos mencionado varias veces que no debemos intentar reinventar la rueda y que debemos reutilizar, repetir y reciclar lo que funciona según la comunidad de software y ML en general. Esto también es cierto para tus arquitecturas de implementación. Cuando discutimos arquitecturas que pueden ser reutilizadas para una variedad de casos de uso diferentes con características similares, a menudo nos referimos a estas como patrones. Usar patrones estándar (o al menos bien conocidos) puede ayudarte realmente a acelerar el tiempo de valor de tu proyecto y a desarrollar tu solución de ML de una manera que sea robusta y extensible.

Dado esto, pasaremos las próximas secciones resumiendo algunos de los patrones arquitectónicos más importantes que se han vuelto cada vez más exitosos en el espacio de ML en los últimos años.

## Swimming in data lakes

El activo más importante para cualquiera que intente usar ML es, por supuesto, los datos que podemos analizar y entrenar nuestros modelos. La era de los grandes datos significó que el tamaño y la variabilidad en el formato de estos datos se convirtieron en un desafío cada vez mayor. Si eres una gran organización (o incluso no tan grande), no es viable almacenar todos los datos que querrás usar para aplicaciones de ML en una base de datos relacional estructurada. Solo la complejidad de modelar los datos para almacenarlos en tal formato sería muy alta. Entonces, ¿qué puedes hacer?

Bueno, este problema se abordó inicialmente con la introducción de los almacenes de datos, que te permiten reunir todo tu almacenamiento de datos relacionales en una sola solución y crear un único punto de acceso. Esto ayuda a aliviar, hasta cierto punto, el problema de los volúmenes de datos, ya que cada base de datos puede almacenar cantidades relativamente pequeñas de datos, incluso si el total es grande.

Estos almacenes de datos fueron diseñados teniendo en cuenta la integración de múltiples fuentes de datos. Sin embargo, siguen siendo relativamente restrictivos, ya que normalmente agrupan la infraestructura para el procesamiento y el almacenamiento. Esto significa que no se pueden escalar muy bien y pueden ser inversiones costosas que generan dependencia del proveedor. Lo más importante para el aprendizaje automático (ML) es que los almacenes de datos no pueden almacenar datos en bruto y datos semiestructurados o no estructurados (por ejemplo, imágenes). Esto descarta automáticamente muchos buenos casos de uso de ML si los almacenes se utilizan como su principal lugar de almacenamiento de datos.

Ahora, con herramientas como Apache Spark, que ya hemos utilizado extensamente a lo largo de este libro, si tenemos los clústeres disponibles, podemos analizar y modelar de manera factible datos de cualquier tamaño o estructura. La pregunta entonces se convierte en, ¿cómo deberíamos almacenarlo?

Los Data lakes son tecnologías que te permiten almacenar cualquier tipo de datos a la escala que necesites. Hay una variedad de proveedores de soluciones de Data lakes, incluidos los principales proveedores de nube pública como Microsoft Azure, Google Cloud Platform (GCP) y AWS. Dado que ya hemos mencionado AWS, centrémonos en eso.

La solución principal de almacenamiento en AWS se llama Simple Storage Service, o S3. Al igual que todas las tecnologías centrales de lagos de datos, puedes cargar efectivamente cualquier cosa en él, ya que se basa en el concepto de almacenamiento de objetos. Esto significa que cada instancia de datos que cargas se trata como su propio objeto con un identificador único y metadatos asociados. Permite que tu bucket S3 contenga simultáneamente fotografías, archivos JSON, archivos .txt, archivos Parquet y cualquier otro número de formatos de datos.

Si trabajas en una organización que no tiene un lago de datos, esto no te excluye automáticamente de hacer aprendizaje automático, pero definitivamente puede hacer que sea un viaje más fácil, ya que siempre sabes cómo puedes almacenar los datos que necesitas para tu problema, sin importar el formato.

## Microservices

La base de código de tu proyecto de aprendizaje automático empezará pequeña, con solo unas pocas líneas al principio. Pero a medida que tu equipo dedique más y más esfuerzo en construir la solución requerida, esto crecerá rápidamente. Si tu solución tiene que tener algunas capacidades diferentes y realizar algunas acciones bastante distintas y mantienes todo esto en la misma base de código, tu solución puede volverse increíblemente compleja.

De hecho, el software en el que los componentes están todos fuertemente acoplados y no son separables como este se llama monolítico, ya que es similar a grandes bloques únicos que pueden existir independientemente de otras aplicaciones. Este tipo de enfoque puede ser adecuado para tu caso de uso, pero a medida que la complejidad de las soluciones sigue aumentando, a menudo se requiere un patrón de diseño mucho más resiliente y extensible.


Las arquitecturas de microservicios son aquellas en las que los componentes funcionales de tu solución están claramente separados, potencialmente en bases de código completamente diferentes o funcionando en infraestructuras completamente diferentes. Por ejemplo, si estamos construyendo una aplicación web orientada al usuario que permite a los usuarios navegar, seleccionar y comprar productos, es posible que tengamos una variedad de capacidades de aprendizaje automático que deseamos implementar en rápida sucesión.

Es posible que queramos recomendar nuevos productos basados en lo que acaban de buscar, puede que queramos recuperar pronósticos de cuándo llegarán los artículos que han pedido recientemente y puede que queramos destacar algunos descuentos de los que creemos que se beneficiarán (basado en nuestro análisis de su comportamiento histórico de cuenta). Esto sería una tarea muy difícil, quizás incluso imposible, para una aplicación monolítica.

La implementación de una arquitectura de microservicios se puede lograr utilizando algunas herramientas, algunas de las cuales cubriremos en la sección sobre Cómo alojar su propio microservicio en AWS. La idea principal es que siempre separe los elementos de su solución en sus propios servicios que no estén fuertemente acoplados entre sí.

Las arquitecturas de microservicios son especialmente buenas para permitir que nuestros equipos de desarrollo logren lo siguiente:

- Depurar, parchear o implementar individualmente los servicios en lugar de desmantelar todo el sistema.
- Evita un único punto de fallo.
- Aumentar la mantenibilidad.
- Permitir que servicios separados sean propiedad de distintos equipos con responsabilidades más claras.
- Acelerar el desarrollo de productos complejos.

Como ocurre con cada patrón de arquitectura o estilo de diseño, no es, por supuesto, una solución mágica, pero sería prudente recordar la arquitectura de microservicios al diseñar nuestra próxima solución.

## Disenos basados en eventos

No siempre quieres operar en lotes programados. Como hemos visto, incluso en la sección anterior, Microservicios, no todos los casos de uso se alinean con ejecutar una gran predicción en lote de un modelo en un horario establecido, almacenando los resultados y luego recuperándolos más tarde. ¿Qué pasa si los volúmenes de datos que necesitas no están disponibles para una ejecución de entrenamiento? ¿Qué sucede si no ha llegado ningún nuevo dato para realizar predicciones? ¿Qué pasaría si otros sistemas pudieran utilizar una predicción basada en puntos de datos individuales en el momento más temprano en que estén disponibles en lugar de a una hora específica todos los días?

En una arquitectura basada en eventos, las acciones individuales producen resultados que luego desencadenan otras acciones individuales en el sistema, y así sucesivamente. Esto significa que los procesos pueden ocurrir tan pronto como puedan y no antes. También permite un flujo de datos más dinámico o estocástico, lo cual puede ser beneficioso si otros sistemas no están funcionando en lotes programados tampoco.

Los patrones basados en eventos podrían mezclarse con otros, por ejemplo, microservicios o procesamiento por lotes. Los beneficios siguen siendo válidos y, de hecho, los componentes basados en eventos permiten una orquestación y gestión más sofisticadas de su solución.

Hay dos tipos de patrones basados en eventos:

- Pub/sub: En este caso, los datos del evento se publican en un intermediario de mensajes o bus de eventos para ser consumidos por otras aplicaciones. En una variante del patrón pub/sub, el intermediario o los buses utilizados están organizados por alguna clasificación adecuada y se designan como temas. Un ejemplo de una herramienta que hace esto es Apache Kafka.
- Event streaming: Los casos de uso de streaming son aquellos en los que queremos procesar un flujo continuo de datos en algo muy cercano al tiempo real. Podemos pensar en esto como trabajar con datos a medida que se mueven a través del sistema. Esto significa que no se persiste en reposo en una base de datos, sino que se procesa a medida que se crea o se recibe por la solución de streaming. Una herramienta de ejemplo para usar en aplicaciones de streaming de eventos sería Apache Storm.


## Batching

Los lotes de trabajo pueden no sonar como el concepto más sofisticado, pero es uno de los patrones más comunes en el mundo del aprendizaje automático. Si los datos que necesitas para la predicción llegan en intervalos de tiempo regulares en lotes, puede ser eficiente programar tus ejecuciones de predicción con una cadencia similar. Este tipo de patrón también puede ser útil si no tienes que crear una solución de baja latencia.

Este concepto también se puede hacer funcionar de manera bastante eficiente por algunas razones:

- Ejecutar en lotes programados significa que sabemos exactamente cuándo necesitaremos recursos de computación, por lo que podemos planificar en consecuencia. Por ejemplo, es posible que podamos apagar nuestros clústeres durante la mayor parte del día o reutilizarlos para otras actividades.
- Los lotes permiten el uso de un número mayor de puntos de datos en tiempo de ejecución, por lo que puedes ejecutar cosas como la detección de anomalías o el agrupamiento a nivel de lote si lo deseas.
- El tamaño de tus lotes de datos a menudo puede elegirse para optimizar algún criterio. Por ejemplo, usar lotes grandes y ejecutar lógica y algoritmos paralelizados en ellos podría ser más eficiente.

Las soluciones de software donde se ejecutan algoritmos de ML en lotes a menudo se parecen mucho a los sistemas clásicos de Extracción, Transformación y Carga (ETL). Estos son sistemas donde los datos se extraen de una fuente o fuentes, antes de ser procesados en su camino hacia un sistema objetivo donde luego se cargan. En el caso de una solución de ML, el procesamiento no es una transformación de datos estándar, como uniones y filtros, sino que es la aplicación de ingeniería de características y tuberías de algoritmos de ML. Es por eso que, en este libro, llamaremos a estos diseños patrones de Extracción, Transformación y Aprendizaje Automático (ETML). ETML se discutirá más en el Capítulo 8, Construyendo un Caso de Uso de Extracción, Transformación y Aprendizaje Automático.


## Containerizing

Si desarrollas software que quieres implementar en algún lugar, que es el objetivo principal de un ingeniero de aprendizaje automático, entonces debes tener muy en cuenta los requisitos ambientales de tu código y cómo diferentes entornos pueden afectar la capacidad de tu solución para ejecutarse. Esto es particularmente importante para Python, que no tiene una capacidad básica para exportar programas como ejecutables independientes (aunque hay opciones para hacerlo). Esto significa que el código Python necesita un intérprete de Python para ejecutarse y debe existir en un entorno Python general donde se hayan instalado las bibliotecas relevantes y los paquetes de soporte.

Una excelente manera de evitar dolores de cabeza desde este punto de vista es hacer la pregunta: ¿Por qué no puedo simplemente poner todo lo que necesito en algo que esté relativamente aislado del entorno host, que pueda enviar y luego ejecutar como una aplicación o programa independiente? La respuesta a esta pregunta es que sí puedes y que haces esto a través de la contenedorización. Este es un proceso mediante el cual una aplicación y sus dependencias pueden ser empaquetadas juntas en una unidad independiente que puede ejecutarse de manera efectiva en cualquier plataforma informática.

La tecnología de contenedores más popular es Docker, que es de código abierto y muy fácil de usar. Aprendamos sobre ella utilizándola para contenerizar una simple aplicación web Flask que podría actuar como una interfaz para un modelo de pronóstico como el creado en la sección Ejemplo 2: API de Pronóstico en el Capítulo 1, Introducción a la Ingeniería de ML.

Las próximas secciones usarán una aplicación Flask simple similar que tiene un punto de servicio de previsión. Como un proxy para un modelo de ML completo, primero trabajaremos con una aplicación esqueleto que simplemente devuelve una lista corta de números aleatorios cuando se solicita una previsión. El código detallado de la aplicación se puede encontrar en el repositorio de GitHub de este libro en https://github.com/PacktPublishing/Machine-Learning-Engineering-with-Python/tree/main/Chapter05/mleip-web-service-main. Los únicos puntos necesarios para la siguiente discusión son que la aplicación Flask devuelve con éxito la salida de la previsión cuando se le consulta en el punto de conexión /forecast.

Ahora, pasamos a discutir cómo contenerizar esta aplicación. Primero, necesitas instalar Docker en tu plataforma siguiendo la documentación en https://docs.docker.com/engine/install/.

1. Una vez que tengas Docker instalado, necesitas indicarle cómo construir la imagen del contenedor, lo cual haces creando un Dockerfile en tu proyecto. El Dockerfile especifica todos los pasos de construcción en texto para que el proceso de construcción de la imagen sea automatizado y fácilmente configurable. Ahora recorreremos la construcción de un ejemplo simple de Dockerfile, que se ampliará en la siguiente sección, Alojando tu propio microservicio en AWS. Primero, necesitamos especificar la imagen base de la que estamos partiendo. Generalmente tiene sentido utilizar una de las imágenes oficiales de Docker como base, así que aquí utilizaremos el entorno python:3.8-slim para mantener las cosas ligeras y eficientes. Esta imagen base se utilizará en todos los comandos que siguen a la palabra clave FROM, que señala que estamos entrando en una etapa de construcción. De hecho, podemos nombrar esta etapa para su uso posterior, llamándola builder utilizando la sintaxis FROM … as:

FROM python:3.8-slim as builder

2. Luego, copiamos todos los archivos que necesitamos del directorio actual a un directorio etiquetado como src en la etapa de construcción e instalamos todos nuestros requisitos utilizando nuestro archivo requirements.txt (si deseas ejecutar este paso sin especificar ningún requisito, puedes simplemente usar un archivo requirements.txt vacío):

**COPY . /src**

**RUN pip install --user --no-cache-dir -r requirements.txt**

3. La siguiente etapa implica pasos similares, pero se denomina mediante la palabra app ya que ahora estamos creando nuestra aplicación. Nota la referencia a la etapa del creador en los pasos 1 y 2 aquí:

**FROM python:3.8-slim as app**

**COPY --from=builder /root/.local /root/.local**
**COPY --from=builder /src .**

4. Podemos definir o añadir a las variables de entorno como estamos acostumbrados en un entorno bash:

**ENV PATH=/root/.local:$PATH**

5. Dado que en este ejemplo vamos a ejecutar una simple aplicación web Flask (más sobre esto más adelante), necesitamos decirle al sistema qué puerto exponer:

**EXPOSE 5000**

6. Podemos ejecutar comandos durante la construcción de Docker utilizando la palabra clave CMD. Aquí, usamos esto para ejecutar app.py, que es el punto de entrada principal de la aplicación Flask, y que iniciará el servicio que llamaremos a través de la API REST para obtener resultados de ML más adelante:

**CMD ["python3", "app.py"]** 

7. Luego podemos construir la imagen con el comando docker build. Aquí, creamos una imagen llamada basic-ml-webservice y le etiquetamos con la etiqueta más reciente:

**docker build -t basic-ml-webservice:latest**

8. Para comprobar que la compilación fue exitosa, ejecuta el siguiente comando en la Terminal:

**docker images --format "table {{.ID}}\t{{.CreatedAt}}\t{{.Repository}}"**

9. Por último, puedes ejecutar tu imagen de Docker con el siguiente comando en tu Terminal:

**docker run basic-ml-webservice:latest**

Ahora que has contenedorizado algunas aplicaciones básicas y puedes ejecutar tu imagen de Docker, necesitamos responder a la pregunta de cómo podemos usar esto para construir una solución de ML alojada en una plataforma adecuada. La siguiente sección cubre cómo podemos hacer esto en AWS.


## Alojar tu propio microservicio en AWS

Una forma clásica de presentar tus modelos de aprendizaje automático es a través de un servicio web ligero alojado en un servidor. Este puede ser un patrón de despliegue muy flexible. Puedes ejecutar un servicio web en cualquier servidor con acceso a internet (aproximadamente) y, si está bien diseñado, a menudo es fácil añadir más funcionalidades a tu servicio web y exponerlo a través de nuevos puntos finales.

En Python, los dos frameworks web más utilizados han sido siempre Django y Flask. En esta sección, nos centraremos en Flask, ya que es el más simple de los dos y se ha escrito extensamente sobre su uso en implementaciones de ML en la web, por lo que podrás encontrar mucho material para construir sobre lo que aprendas aquí.

En AWS, una de las maneras más simples de alojar tu solución web Flask es como una aplicación en contenedor en una plataforma adecuada. Vamos a repasar los conceptos básicos de esto aquí, pero no dedicaremos tiempo a los aspectos detallados de mantener una buena seguridad web para tu servicio. Discutir esto en su totalidad podría requerir un libro completo por sí mismo, y hay recursos excelentes y más enfocados en otros lugares.

Asumiremos que tienes tu cuenta de AWS configurada desde el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático. Si no la tienes, vuelve atrás y refresca lo que necesitas hacer. Necesitaremos la Interfaz de Línea de Comandos (CLI) de AWS. Esto se puede instalar en un sistema Linux x86_64 con los siguientes comandos:

curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install

Puedes encontrar los comandos apropiados para instalar y configurar el AWS CLI, así como mucha otra información útil, en las páginas de documentación de AWS CLI en https://docs.aws.amazon.com/cli/index.html.

En el siguiente ejemplo, utilizaremos el Registro de Contenedores Elástico de Amazon (ECR) y el Servicio de Contenedores Elástico (ECS) para alojar una aplicación web esquelética containerizada. En el Capítulo 7, Construyendo un Microservicio Ejemplo de ML, detallaremos el modelo de ML para completar la solución de ingeniería de ML.

Desplegar nuestro servicio en ECS requerirá algunos componentes diferentes, que revisaremos en las próximas secciones:

- Nuestro contenedor alojado dentro de un repositorio en ECR
- Un clúster y un servicio creados en ECS
- Un equilibrador de carga de aplicaciones creado a través del servicio de Elastic Compute Cloud (EC2)

Primero, abordemos el envío del contenedor a ECR.


# Pushing to ECR

Veamos los siguientes pasos:

1. Tenemos el siguiente Dockerfile definido dentro del directorio del proyecto de la sección de Containerizar:

FROM python:3.8-slim as builder

COPY . /src

RUN pip install --user --no-cache-dir -r requirements.txt

FROM python:3.8-slim as app

COPY --from=builder /root/.local /root/.local

COPY --from=builder /src .

ENV PATH=/root/.local:$PATH

EXPOSE 5000

CMD ["python3", "app.py"]

2. Luego podemos usar la CLI de AWS para crear un repositorio ECR para alojar nuestro contenedor. Llamaremos al repositorio basic-ml-microservice y estableceremos la región como eu-west-1, pero esto debería cambiarse a la región que parezca más adecuada:

aws ecr create-repository \

--repository-name basic-ml-microservice \

--image-scanning-configuration scanOnPush=true \

--region eu-west-1

3. Luego podemos iniciar sesión en el registro de contenedores con el siguiente comando en la Terminal:

aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-

stdin <YOUR_AWS_ID>.dkr.ecr.eu-west-1.amazonaws.com/basic-ml-microservice

4. Luego, si navegamos al directorio que contiene el Dockerfile (app), podemos ejecutar el siguiente comando para construir el contenedor:

docker build --tag basic-ml-microservice .

5. El siguiente paso etiqueta la imagen:

docker tag flask-docker-demo-app:latest <YOUR_AWS_ID>.dkr.ecr.eu-west-1.amazonaws.com/basic-ml-microservice:latest

6. Luego desplegamos la imagen de Docker que hemos construido recientemente en el registro de contenedores con el siguiente comando:

docker push <YOUR_AWS_ID>.dkr.ecr.eu-west-1.amazonaws.com/basic-ml-microservice:latest

En la siguiente sección, configuramos nuestro clúster en ECS.

# Hosting on ECS

¡Ahora, comencemos con la configuración!

1. Luego creamos un clúster de ECS con la configuración de EC2 Linux + Redes. Para hacer esto, primero navega a ECS en la Consola de Administración de AWS y haz clic en Crear Clúster:

2. Luego, en el siguiente paso, selecciona EC2 Linux + Redes:

3. Luego se nos dan opciones para configurar nuestro clúster. Aquí, llamamos al clúster mleip-web-app-demos (¡o cualquier nombre que nos guste!) y especificamos que las instancias de EC2 utilizadas sean bajo demanda, ya que este es un microservicio en el que probablemente no queramos permitir tiempos de inactividad mientras esperamos las instancias de spot. Seleccionamos una máquina relativamente pequeña como tipo de instancia, una t2.micro, que es buena para computación ligera y de propósito general, y se adapta bastante bien a un pequeño servicio de predicción de ML. Dado que esto es una demostración, solo necesitamos una instancia y podemos usar las configuraciones predeterminadas para el resto de los pasos:

4. El siguiente paso es configurar la red para nuestro clúster. Podemos optar por crear una nueva Nube Privada Virtual (VPC) y grupos de seguridad para esta VPC con configuraciones apropiadas o reutilizar las ya definidas anteriormente. Hay mucha más información sobre las VPC en AWS en https://docs.amazonaws.cn/en_us/vpc/latest/userguide/what-is-amazon-vpc.html:

5. Una vez que hayamos hecho esto, si hacemos clic en Crear Clúster en la parte inferior de la página, seremos dirigidos a una página de estado que muestra el progreso de la instanciación del clúster:

6. Una vez completado, si navegamos de vuelta a la página de ECS, podemos ver una vista resumen del clúster que acabamos de crear, como en la Figura 5.10:

7. Ahora necesitamos crear una definición de tarea en el clúster. Hacemos esto seleccionando Definiciones de tarea en el menú de la izquierda y luego seleccionando Crear nueva definición de tarea.

8. Luego seleccionamos el tipo de compatibilidad de Fargate para el clúster, lo que nos ayuda a gestionar gran parte de la infraestructura de backend sin pensar en ello y solo pagar por los recursos computacionales que utilizamos:

9. Luego completamos algunos detalles para la definición de la tarea, como se muestra en la Figura 5.13. Por ejemplo, llamaremos a la definición de la tarea microservice-forecast-task y usaremos un rol de tarea creado en la Consola de Administración de AWS llamado ecsTaskExecutionRole. Para más detalles sobre los roles de Gestión de Acceso e Identidad (IAM), por favor consulte https://aws.amazon.com/iam/.

10. El siguiente paso es definir el tamaño de la tarea. Aquí seleccionamos los valores más bajos de 0.5 GB y 0.25 vCPU:

11. Ahora, agregamos un contenedor a la configuración haciendo clic en la opción Agregar contenedor en la página y luego llenando los detalles del nombre del contenedor y la URI de la imagen. También podemos agregar límites de memoria a la tarea y proporcionar el puerto del contenedor que deseamos exponer al tráfico de internet entrante (aquí usaremos el puerto 5000). Esto se muestra en la Figura 5.15. Estos son los mismos nombres de contenedor y URIs de imagen que usamos para subir el contenedor a ECR:

12. Después de completar el paso anterior, podemos seleccionar Crear en la parte inferior de la página para crear la tarea. Una creación exitosa significa que deberíamos poder ver la nueva definición de tarea en el servicio ECS cuando seleccionamos Definiciones de Tarea en el lado izquierdo.

Ahora, el paso final para configurar nuestra solución alojada en ECS es la creación de un servicio. Ahora explicaremos cómo hacerlo:

1. Primero, navega de vuelta a la página de ECS y luego a la sección de Clústeres. Selecciona el clúster mleip-web-app-demos y luego el botón Crear Servicio. A continuación, se nos presenta una página que solicita varios valores de configuración. En este ejemplo, seleccionaremos el Tipo de Lanzamiento como Fargate y luego daremos el nombre del clúster y la definición de tarea para coincidir con los nombres de los elementos que acabamos de crear en los pasos anteriores.

2. En la misma página de configuración, también tenemos que definir algunos valores sobre cómo se escalará, monitorizará y desplegará el servicio, como se muestra en la Figura 5.18. Para esta demostración sencilla, podemos crear un tipo de servicio REPLICA que instancia dos tareas. Esto significa efectivamente que tendrá dos instancias de su aplicación en contenedor disponibles en cualquier momento para ayudar a construir algo de redundancia. También definimos el porcentaje mínimo y máximo de nuestras tareas, que deberían estar sanas y en ejecución para este servicio durante nuestro despliegue de tipo Rolling Update:

3. Luego hay una sección sobre balanceo de carga que debemos completar. Aquí, selecciona Application Load Balancer y luego selecciona un balanceador de carga apropiado si ya tienes uno definido. Si no, puedes usar la siguiente sección sobre la Creación de un Balanceador de Carga para ayudar. Luego seleccionamos la solución de contenedor que deseamos equilibrar, que aquí es nuestra solución basic-ml-microservice con el puerto 5000 expuesto. La misma sección también requiere la definición de nuevos puertos de escucha y lo que se conoce como un nombre de grupo de destino. Un grupo de destino es utilizado por tu balanceador de carga para enrutar solicitudes a los objetivos de endpoint apropiados. Podemos crear uno nuevo aquí, pero necesitamos asegurarnos de que nuestro balanceador de carga lo conozca (ver la siguiente sección), así que puede que necesites actualizar el balanceador de carga. Finalmente, definimos un patrón de ruta para el oyente, que aquí debemos asegurarnos que termine en /forecast ya que esta es la ruta predeterminada que utiliza nuestra aplicación Flask. Todas estas asignaciones de valor se muestran en la Figura 5.20:

4. Ahora puedes seleccionar Crear Servicio y relajarte felizmente ya que acabas de implementar un pequeño microservicio ordenado. Puedes probar esto llamando a tu aplicación a través de Postman. La URI que necesitas llamar tiene su primera parte determinada por el DNS de tu balanceador de carga de aplicación (aquí comenzando http://mleip-app-lb), que puedes encontrar en el servicio de EC2. El resto de la URI está definido por el endpoint requerido definido en tu aplicación Flask, aquí, /forecast. Llamar a la solución da el resultado mostrado en la Figura 5.21:



# Creating a load balancer

En la sección de Hosting en ECS, utilizamos un balanceador de carga llamado mleip-app-lb. Esta sección te ayudará a entender cómo crear esto si no lo has hecho antes en AWS. Los balanceadores de carga son piezas de software que enrutan de manera eficiente el tráfico de red, como las solicitudes HTTP entrantes, entre múltiples servidores para garantizar la estabilidad del servicio. Son una parte importante de cualquier arquitectura que atiende dinámicamente las solicitudes.

Afortunadamente, AWS hace que la creación de equilibradores de carga sea relativamente fácil, como veremos a continuación. Los próximos pasos describirán cómo crear un equilibrador de carga de aplicaciones que se puede utilizar en la creación de varias aplicaciones web diferentes:

1. Primero, navegamos a la Consola de Gestión de EC2 en AWS, luego seleccionamos Balanceadores de Carga en el menú de la izquierda y finalmente hacemos clic en Crear Balanceador de Carga. Luego deberíamos recibir varias opciones para diferentes tipos de balanceador de carga. Aquí, seleccionaremos Balanceador de Carga de Aplicación, como en la Figura 5.22:

2. El siguiente paso es configurar el balanceador de carga. Aquí, le daremos el nombre de mleip-app-lb, que coincide con el balanceador de carga utilizado en la sección de Hosting en ECS. También añadimos un oyente utilizando el protocolo HTTP en el puerto 80. Es importante que el oyente creado aquí esté en el mismo puerto seleccionado cuando proporcionamos la información de balanceo de carga para la creación de nuestro servicio ECS.

Finalmente, seleccionamos las zonas de disponibilidad de VPC, que nuevamente deben coincidir con las zonas de VPC aplicables a nuestro servicio ECS.

3. A continuación, configuramos la seguridad para el balanceador de carga. Si solo hemos creado un oyente en el protocolo HTTP (como hicimos en el Paso 2), entonces recibiremos una advertencia sobre no tener un oyente seguro en la siguiente página, la cual puedes aliviar utilizando el protocolo HTTPS. Esto requiere la gestión de certificados SSL, que no cubriremos aquí. Para más información, consulta https://docs.aws.amazon.com/elasticloadbalancing/latest/application/create-https-listener.html. Por ahora, podemos avanzar al siguiente paso con el protocolo HTTP en su lugar, y crear un nuevo grupo de seguridad o seleccionar uno existente, como se muestra en la Figura 5.24:

4. El siguiente paso requiere que configuremos el enrutamiento de las solicitudes a través del equilibrador de carga. Esto se hace seleccionando o creando un grupo de destino (ya discutido en la sección de Alojamiento en ECS) con el protocolo de enrutamiento adecuado y las rutas de verificación de salud (ver Figura 5.25). Creamos un nuevo grupo de destino aquí y luego lo editamos más tarde para asegurar el enrutamiento al servicio adecuado según sea necesario:

5. Navegar al siguiente paso en el proceso solicita que registres destinos, pero esto se puede omitir por ahora y los destinos se pueden registrar más tarde seleccionando Agregar Listener a tu balanceador de carga una vez que se haya creado. Luego podemos navegar a la página final, que muestra una revisión del balanceador de carga que estás a punto de crear. Esto se verá similar a la Figura 5.26. Selecciona Crear para completar el proceso:

6. La confirmación de la creación exitosa del balanceador de carga debería aparecer en la página, como se muestra en la Figura 5.27:

¡Y eso es todo! Ahora tenemos todas las piezas en su lugar y conectadas para tener un microservicio funcional adecuado para servir modelos de ML. La siguiente sección pasará a discutir cómo podemos utilizar herramientas de canalización listas para producción para desplegar y orquestar nuestros trabajos de ML.


# Pipelining 2.0

En el Capítulo 4, Empaquetando, discutimos los beneficios de escribir nuestro código de ML como tuberías. Hablamos sobre cómo implementar algunas tuberías básicas de ML utilizando herramientas como sklearn y Spark MLlib. Las tuberías que nos interesaban allí eran formas muy agradables de simplificar nuestro código y hacer que varios procesos estuvieran disponibles para usar dentro de un solo objeto para simplificar una aplicación.

Sin embargo, todo lo que discutimos entonces estaba muy enfocado en un solo archivo de Python y no necesariamente en algo que pudiéramos extender con mucha flexibilidad fuera de las limitaciones del paquete que estábamos usando. Con las técnicas que discutimos, por ejemplo, sería muy difícil crear pipelines donde cada paso estuviera usando un paquete diferente o incluso donde fueran programas totalmente diferentes. No nos permitieron construir mucha sofisticación en nuestros flujos de datos o lógica de aplicación tampoco, ya que si uno de los pasos fallaba, el pipeline fallaba, y eso era todo.

Las herramientas que estamos a punto de discutir llevan estos conceptos al siguiente nivel. Te permiten gestionar los flujos de trabajo de tus soluciones de ML para que puedas organizar, coordinar y orquestar elementos con el nivel apropiado de complejidad para completar la tarea.

## Airflow

Apache Airflow es la herramienta de gestión de flujos de trabajo que fue desarrollada inicialmente por Airbnb en la década de 2010 y ha sido de código abierto desde su inicio. Proporciona a científicos de datos, ingenieros de datos e ingenieros de ML la capacidad de crear programáticamente canales complejos a través de scripts en Python. La gestión de tareas de Airflow se basa en la definición y luego en la ejecución de un Grafo Acíclico Dirigido (DAG) con nodos como las tareas que se deben ejecutar. Los DAGs también se utilizan en TensorFlow y Spark, así que es posible que hayas oído hablar de ellos antes.

Airflow contiene una variedad de operadores predeterminados que le permiten definir DAGs que pueden llamar y usar múltiples componentes como tareas, sin preocuparse por los detalles específicos de esa tarea. También proporciona funcionalidad para programar sus tuberías:

1. Como ejemplo, construyamos un pipeline de Apache Airflow que obtendrá datos, realizará ingeniería de características, entrenará un modelo y luego lo persistirá en MLFlow. No cubriremos la implementación detallada de cada comando, sino que simplemente te mostraremos cómo se ensamblan en un DAG de Airflow. Primero, importamos los paquetes relevantes de Airflow y cualquier paquete de utilidad que necesitemos:

In [ ]:
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.utils.dates import days_ago

2. A continuación, Airflow te permite definir argumentos predeterminados que pueden ser referenciados por todas las tareas siguientes, con la opción de sobrescribir en el mismo nivel:

In [ ]:
default_args = {
'owner': 'Andrew McMahon',
'depends_on_past': False,
'start_date': days_ago(31),
'email': ['example@example.com'],
'email_on_failure': False,
'email_on_retry': False,
'retries': 1,
'retry_delay': timedelta(minutes=2)
}

3. Luego tenemos que instanciar nuestro DAG y proporcionar los metadatos relevantes, incluyendo nuestro intervalo de programación:

In [ ]:
dag = DAG(
'classification_pipeline',
default_args=default_args,
description='Basic pipeline for classifying the Wine Dataset',
schedule_interval=timedelta(days=1), # run daily? check
)

Luego, lo único que se requiere es definir tus tareas. Aquí, definimos una tarea inicial que ejecuta un script de Python que obtiene nuestro conjunto de datos:

In [ ]:
get_data = BashOperator(
task_id='get_data',
bash_command='python3 /usr/local/airflow/scripts/get_data.py',
dag=dag,
)

A continuación, realizamos una tarea que toma estos datos y lleva a cabo nuestros pasos de entrenamiento del modelo. Esta tarea podría, por ejemplo, encapsular uno de los tipos de pipeline que cubrimos en el Capítulo 3, De Modelo a Fábrica de Modelos; por ejemplo, un pipeline de Spark MLlib:

In [ ]:
train_model= BashOperator(
task_id='train_model',
depends_on_past=False,
bash_command='python3 /usr/local/airflow/scripts/train_model.py',
retries=3,
dag=dag,
)

6. El paso final de este proceso tomará el modelo entrenado resultante y lo publicará en MLFlow. Esto significa que otros servicios o pipelines pueden utilizar este modelo para hacer predicciones.

In [ ]:
persist_model = BashOperator(
task_id='persist_model',
depends_on_past=False,
bash_command='python /usr/local/airflow/scripts /persist_model.py,
retries=3,
dag=dag,
)

7. Finalmente, definimos el orden de ejecución de los nodos de tarea que hemos definido en el DAG utilizando el operador >>. Las tareas anteriores podrían haberse definido en cualquier orden, pero la siguiente sintaxis estipula cómo deben ejecutarse:

get_data >> train_model >> persist_model

En las próximas secciones, cubriremos brevemente cómo configurar un pipeline de Airflow en AWS utilizando principios de CI/CD. Esto reunirá parte de la configuración y trabajo que hemos estado realizando en capítulos anteriores del libro.

## Airflow on AWS

AWS ofrece un servicio alojado en la nube llamado Workflows Administrados para Apache Airflow (MWAA) que te permite implementar y alojar tus pipelines de Airflow de manera fácil y robusta. Aquí, cubriremos brevemente cómo hacerlo.

Luego completas los siguientes pasos:

1. Seleccione Crear entorno en la página de inicio de MWAA.
2. A continuación, se le proporcionará una pantalla que solicita los detalles de su nuevo entorno Airflow. La figura 5.29 muestra los pasos de alto nivel que el sitio web le guía a través de Los detalles del entorno, como se muestra en la Figura 5.30, es donde especificamos el nombre de nuestro entorno. Aquí lo hemos llamado mleip-airflow-dev-env:
3. Para que MWAA funcione, necesita poder acceder al código que define el DAG y a cualquier archivo de requisitos o complementos asociados. El sistema luego solicita un bucket de AWS S3 donde residen estas piezas de código y configuración. En este ejemplo, creamos un bucket llamado mleip-airflow-example que contendrá estas piezas.
4. Luego tenemos que definir la configuración de la red que usará la instancia administrada de Airflow. Esto puede ser un poco confuso si eres nuevo en redes, así que podría ser bueno leer sobre los temas de subredes, direcciones IP y VPCs. Crear un nuevo VPC de MWAA es el enfoque más fácil para comenzar en términos de redes aquí, pero tu organización tendrá especialistas en redes que pueden ayudarte a usar la configuración adecuada para tu situación. Iremos con esta ruta más simple y haremos clic en Crear VPC de MWAA, lo que abre una nueva ventana donde podemos configurar rápidamente un nuevo VPC y una configuración de red basada en una definición de pila estándar proporcionada por AWS, como se muestra en la Figura 5.33:
6. A continuación, tenemos que definir la clase de Entorno que queremos iniciar. Actualmente, hay tres opciones. Aquí, usamos la más pequeña, pero puedes elegir el entorno que mejor se adapte a tus necesidades (¡siempre pide permiso al pagador de la factura!). La Figura 5.35 muestra que podemos seleccionar la clase de entorno mw1.small con un recuento de trabajadores de 1 a 10. MWAA te permite cambiar la clase de entorno después de instanciarla, si es necesario, por lo que a menudo puede ser mejor comenzar con poco y escalar según sea necesario desde el punto de vista del costo:
7. Ahora, si se desea, confirmamos algunos parámetros de configuración opcionales (o dejamos estos en blanco, como se hace aquí) y confirmamos que estamos satisfechos de que AWS cree y use un nuevo rol de ejecución. La figura 5.36 muestra un ejemplo de esto (y no te preocupes, el grupo de seguridad ya habrá sido eliminado para cuando estés leyendo esta página).

8. La siguiente página te proporcionará un resumen final antes de permitirte crear tu entorno de MWAA. Una vez que hagas esto, podrás ver en el servicio de MWAA tu entorno recién creado, como en la Figura 5.37. Este proceso puede tardar un tiempo, y para este ejemplo tomó alrededor de 30 minutos.


Ahora que tienes este entorno MWAA y has proporcionado tu DAG al bucket S3 al que apunta, puedes abrir la interfaz de usuario de Airflow y ver los trabajos programados definidos por tu DAG. Ahora has desplegado un servicio básico en funcionamiento sobre el cual podemos construir en trabajos futuros.

`NOTA IMPORTANTE`: Una vez que hayas creado este entorno MWAA, no puedes pausarlo, ya que cuesta una pequeña cantidad por hora (alrededor de 0.5 USD por hora para la configuración del entorno anterior). MWAA actualmente no contiene una función para pausar y reanudar un entorno, por lo que tendrás que eliminar el entorno y volver a instanciar uno nuevo con la misma configuración cuando sea necesario. Esto se puede automatizar utilizando herramientas como Terraform o AWS CloudFormation, que no cubriremos aquí. Así que, una advertencia: NO DEJES ACCIDENTALMENTE TU ENTORNO EN MARCHA. Por ejemplo, definitivamente no lo dejes funcionando durante una semana, como yo podría haberlo hecho o no.

## Revisitar CI/CD

Introdujimos los conceptos básicos de CI/CD en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático, y discutimos cómo se puede lograr esto utilizando GitHub Actions. Ahora daremos un paso más allá y comenzaremos a configurar pipelines de CI/CD que desplieguen código en la nube.

Primero, comenzaremos con un ejemplo importante donde enviaremos algo de código a un bucket de AWS S3. Esto se puede hacer creando un archivo yml en tu repositorio de GitHub bajo el directorio .github/workflows llamado aws-s3-deploy.yml. Este será el núcleo alrededor del cual formaremos nuestra pipeline de CI/CD.

Continuar ...

# Resumen

En este capítulo, hemos discutido algunos de los conceptos más importantes a la hora de desplegar tus soluciones de ML. En particular, nos centramos en los conceptos de arquitectura y qué herramientas podríamos utilizar potencialmente al desplegar soluciones en la nube. Cubrimos algunos de los patrones más importantes utilizados en la ingeniería de ML moderna y cómo estos se pueden implementar con herramientas como contenedores y AWS Elastic Container Registry y Elastic Container Service, así como cómo crear tuberías programadas en AWS utilizando Managed Workflows para Apache Airflow. También exploramos cómo conectar el ejemplo de MWAA con GitHub Actions, de modo que los cambios en tu código puedan activar directamente actualizaciones de los servicios en ejecución, proporcionando una plantilla para usar en futuros procesos de CI/CD.

En el próximo capítulo, analizaremos la cuestión de cómo escalar nuestras soluciones para poder manejar grandes volúmenes de datos y cálculos de alto rendimiento.
